Quadratic programming problem solution using kkt conditions
* kkt conditions 

In [8]:
import pandas
import math
import cv2
import math
import numpy as np
import copy

In [9]:
A = [[2, -2, 2, 1, -1, 0],[-2, 4, 1, -4, 0, -1],[2, 1, 0, 0, 0, 0], [1, -4, 0, 0, 0, 0]]
B = [4, 0, 6, 0]
relations = [0, 0, -1, -1]
X = [0, 1]
Y = [6, 7]
lambda_ = [2, 3]
theta = [4, 5]
cost_vector = [0, 0, 0, 0, 0, 0]
init_simplex(A, B, cost_vector, relations, X, Y, lambda_, theta)





iteration 1
++++++++++++++++
A
[[ 2 -2  2  1 -1  0  1  0  0  0]
 [-2  4  1 -4  0 -1  0  1  0  0]
 [ 2  1  0  0  0  0  0  0  1  0]
 [ 1 -4  0  0  0  0  0  0  0  1]]
B
[4 0 6 0]
present basic vars: [6, 7, 8, 9] values: [4, 0, 6, 0]
Entering: 1, leaving: 7
pivot: 4
max profit for this round: 1.0



iteration 2
++++++++++++++++
A
[[ 1.    0.    2.5  -1.   -1.   -0.5   1.    0.5   0.    0.  ]
 [-0.5   1.    0.25 -1.    0.   -0.25  0.    0.25  0.    0.  ]
 [ 2.5   0.   -0.25  1.    0.    0.25  0.   -0.25  1.    0.  ]
 [-1.    0.    1.   -4.    0.   -1.    0.    1.    0.    1.  ]]
B
[4. 0. 6. 0.]
present basic vars: [6, 1, 8, 9] values: [4.0, 0.0, 6.0, 0.0]
Entering: 0, leaving: 8
pivot: 2.5
max profit for this round: 2.6



iteration 3
++++++++++++++++
A
[[ 0.   0.   2.6 -1.4 -1.  -0.6  1.   0.6 -0.4  0. ]
 [ 0.   1.   0.2 -0.8  0.  -0.2  0.   0.2  0.2  0. ]
 [ 1.   0.  -0.1  0.4  0.   0.1  0.  -0.1  0.4  0. ]
 [ 0.   0.   0.9 -3.6  0.  -0.9  0.   0.9  0.4  1. ]]
B
[1.6 1.2 2.4 2.4]
prese

#### initializes the simplex
* A, B: are the n*m and n*1 matrices
* cost_vector: (n*1 vector) is the coeff in the cost function for the n variables
* relations: (m*1 vector) Represents the in/equality of the m equations. -1 if less than equal to, +1 if greater than equal to, 0 if equal to
* X, Y, lambda_, theta: dictionary for the var type. Eg. X[i]=j keys = ith X variable, value = jth column or var_id 

In [2]:
def init_simplex(A, B, cost_vector, relations, X, Y, lambda_, theta):

## Determining the in/equalities for the synthetic variables
    rel_count = {-1:0, 0:0, 1:0}
    A_ori = copy.deepcopy(A)
    for rel_i in range(0,len(relations)):
        if (relations[rel_i] == -1):
            rel_count[-1] += 1
        elif (relations[rel_i] == 1):
            rel_count[1] += 1
        elif (relations[rel_i] == 0):
            rel_count[0] += 1
        
        
    
    num_ori_var = len(A[0])
    num_eqn = len(A)

## simple case when the constraints are of the type Ax<=B.
    if (rel_count[-1] == len(A)):
#Adding slack variable
        for row_i in range(0,len(A)):
            for push_i in range(0,len(A)):
                if (push_i == row_i):
                    A[row_i].append(1)
                else:
                    A[row_i].append(0)

        num_var = len(A[0])

#Adding coeff to the cost_vector for the slack variables
        for slack_i in range(0,len(A)):
            cost_vector.append(0)

#defining indices for the initial basic variable.
        basic_var_i = []
        for i in range(num_ori_var, len(A[0])):
            basic_var_i.append(i)
        
        basic_var = calc_basic_var(basic_var_i, A, B)
        var_dict, cost = simplex_solver(A, B, cost_vector, basic_var, basic_var_i)


## when the constraints are not just of the less than types
    else:
        slack_var_neg = {}
        slack_var_pos = {}
        art_var = {}
        write_i = num_ori_var
        num_syn_var = rel_count[-1] + rel_count[0] + rel_count[1]*2
        
        for row_i in range(0,len(A)):
            for i in range (num_ori_var, write_i):
                A[row_i].append(0)
##Adding the positive coef slack var
            if (relations[row_i]==-1):
                A[row_i].append(1)
                slack_var_pos[row_i] = write_i
                write_i += 1
                for i in range (write_i, num_syn_var+num_ori_var):
                    A[row_i].append(0)
##Adding the neg coeff slack var and artificial var
            elif (relations[row_i]==1):
                A[row_i].append(-1)
                slack_var_neg[row_i] = write_i
                A[row_i].append(1)
                art_var[row_i] = write_i+1
                write_i += 2
                for i in range (write_i, num_syn_var+num_ori_var):
                    A[row_i].append(0)
                    # print ("+")
##Adding the artificial var
            elif (relations[row_i]==0):
                A[row_i].append(1)
                art_var[row_i] = write_i
                write_i += 1
                for i in range (write_i, num_syn_var+num_ori_var):
                    A[row_i].append(0)
        
##Creating the list of diff types of slack variables

        Y = list(slack_var_pos.values()) + list(slack_var_neg.values())
        Y.sort()
        art_cost_fn = []

##Creating the artificial cost functions. To force the artificial variables to become zero!
## maximize sum (-z_i) for all i
        for var_i in range(0, num_ori_var+num_syn_var):
            if ((var_i in art_var.values())):
                art_cost_fn.append(-1)
            else:
                art_cost_fn.append(0)
        
        art_var_i = list(art_var.values())+list(slack_var_pos.values())
        art_var_i.sort()

        var_dict, cost, A_trans, B_trans = simplex_solver(A, B, art_cost_fn, B, art_var_i, X, Y, lambda_, theta)

In [3]:
def report_max (lst, X, Y, lambda_, theta, basic_var_i):
    highest_i = 0
    while (highest_i < len(lst)):
        max_val = n_largest(lst, highest_i)
        max_i = lst.index(max_val)
        
        if (max_i in list(X)):
            pos = X.index(max_i)
            if (theta[pos] in basic_var_i):
                highest_i += 1
            else:
                break
        elif (max_i in list(Y)):
            pos = Y.index(max_i)
            if (lambda_[pos] in basic_var_i):
                highest_i += 1
            else:
                break

        elif (max_i in list(lambda_)):
            pos = lambda_.index(max_i)
            if (Y[pos] in basic_var_i):
                highest_i += 1
            else:
                break

        elif (max_i in list(theta)):
            pos = theta.index(max_i)
            if (X[pos] in basic_var_i):
                highest_i += 1
            else:
                break
        
    if (len(lst) == 0):
        return (None, None)

    return (max_val, max_i)

In [4]:
def simplex_solver (A, B, cost_vector, basic_var, basic_var_i, X, Y, lambda_, theta):

    num_eqn = len(A)
    
    num_var = len (A[0])

    cost_z = calc_z(num_var, num_eqn, basic_var_i, A, cost_vector)

    cost_c_z = calc_cost_c_z(cost_z, cost_vector)

    max_c_z, entering_var = report_max(cost_c_z, X, Y, lambda_, theta, basic_var_i)
    
    if (entering_var == None):
        print ("Infeasible LP")
        return (None, None, None, None)

    itr = 1
    finished = False
    while (max_c_z > 0):
        print('\n\n')
        print ("iteration {}".format(itr))
        print ("++++++++++++++++")
        print ("A")
        print (np.array(A))
        print ("B")
        print (np.array(B))
        print ("present basic vars: "+str(basic_var_i)+" values: "+str(basic_var))
        
                
        leaving_row = -1
        for row_i in range(0, num_eqn):
            if (A[row_i][entering_var]>0 and B[row_i]/A[row_i][entering_var]>=0):
                if (leaving_row == -1):
                    leaving_row = row_i
                elif (B[row_i]/A[row_i][entering_var] <= B[leaving_row]/A[leaving_row][entering_var]):
                    leaving_row = row_i
        if (leaving_row==-1):
            print ("Unconstrained LPP")
            break

        leaving_var_i = basic_var_i[leaving_row]
        print ("Entering: {}, leaving: {}".format(entering_var, leaving_var_i))
        A,B = pivot(A, B, leaving_row, entering_var)
        
        for i in range (0, len(basic_var_i)):
            if (leaving_var_i == basic_var_i[i]):
                basic_var_i[i] = entering_var
        
        basic_var = calc_basic_var(basic_var_i, A, B)
        
        cost_z = calc_z(num_var, num_eqn, basic_var_i, A, cost_vector)
        cost_c_z = calc_cost_c_z(cost_z, cost_vector)
        max_c_z, entering_var = report_max(cost_c_z, X, Y, lambda_, theta, basic_var_i)
        print('max profit for this round: {}'.format(max_c_z))
        if max_c_z <= 0:
            finished = True
        
 
        cost = []
        for var_i in range(0,num_var):
            if (var_i in basic_var_i):
                for i in range(0,len(basic_var_i)):
                    if (basic_var_i[i]==var_i):
                        cost.append(basic_var[i]*cost_vector[var_i])
        
        print ("==================")
        var_dict = {}
        for i in range(0, len(basic_var_i)):
            var_dict[basic_var_i[i]] = basic_var[i]
            
        itr += 1
    
    print ('final basis')
    print (str(basic_var_i) +" with values "+str(basic_var))
    print("simplex finished with final values")
    for i in X:
        print("x_{} : {}".format(i+1, basic_var[i+1]))
    return (var_dict, cost, A, B)


In [5]:
def n_largest(lst, n):
    temp = copy.deepcopy(lst)
    temp.sort()
    return (temp[len(temp)-n-1])

In [6]:
       

def pivot(A, B, leaving_var_i, entering_var):
    pivot_val = A[leaving_var_i][entering_var]
    print('pivot: {}'.format(pivot_val))
    A[leaving_var_i] = [x/pivot_val for x in A[leaving_var_i]]
    B[leaving_var_i] = B[leaving_var_i]/pivot_val
    for row_i in (range(0,len(A))):
        if (row_i == leaving_var_i):
            continue
        B[row_i] = B[row_i] - B[leaving_var_i]*(A[row_i][entering_var])
        if (B[row_i]<0):
            print ("B turning negative")
        temp_row = [x*A[row_i][entering_var] for x in A[leaving_var_i]]
        A[row_i] = [x-y for (x,y) in zip(A[row_i],temp_row)]
    return (A,B)

def calc_basic_var(basic_var_i, A, B):
    basic_var = []
    for var_i in basic_var_i:
        for row_i in range(0, len(A)):
            if (A[row_i][var_i]==1):
                basic_var.append(B[row_i])     
    return (basic_var)


def calc_z(num_var, num_eqn, basic_var_i, A, cost_vector):
    cost_z = [0] * num_var

    for var_i in range(0,num_var):
        sum_i = 0
        for row_i in range(0, num_eqn):
            sum_i += (cost_vector[basic_var_i[row_i]]*A[row_i][var_i])
        cost_z[var_i] = sum_i
    return cost_z

#calculating c_i - z_i for all variables
def calc_cost_c_z(cost_z, cost_vector):
    cost_c_z = []
    for var_i in range(0, len(cost_z)):
        cost_c_z.append(cost_vector[var_i] - cost_z[var_i])
    return cost_c_z